In [ ]:
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# ---------------------------------------------------------
# 1. Settings
# ---------------------------------------------------------

# Folder that contains your CSVs (one per location)
data_dir = "../data_for_prediction/"  # <-- CHANGE THIS

# Pattern to find all station files
file_pattern = str(Path(data_dir) / "*.csv")
files = glob.glob(file_pattern)

# Targets (horizons) we care about
targets = ["prec_1d_ahead", "prec_3d_ahead", "prec_7d_ahead"]

# Moving-average window (k)
MA_WINDOW = 3

# ---------------------------------------------------------
# 2. Helper: compute baselines for one location
# ---------------------------------------------------------

def make_baseline_predictions_for_file(csv_path, targets, ma_window=3):
    """
    Compute persistence (t-1) and moving-average baselines
    for one CSV (one location).

    Returns a DataFrame with columns:
      ['location', 'date', 'target', 'model', 'y_true', 'y_pred']
    """
    df = pd.read_csv(csv_path)

    # Build datetime column: rename YYYY/MM/DD to year/month/day for to_datetime
    df["date"] = pd.to_datetime(
        df[["YYYY", "MM", "DD"]].rename(columns={
            "YYYY": "year",
            "MM": "month",
            "DD": "day"
        })
    )

    # Location id from filename (without directory and extension)
    location_id = Path(csv_path).stem

    preds_list = []

    # --- Persistence baseline: y_hat = today's 'prec' ---
    for tgt in targets:
        y_true = df[tgt]
        y_pred = df["prec"]  # same for all horizons

        tmp = pd.DataFrame({
            "location": location_id,
            "date": df["date"],
            "target": tgt,
            "model": "persistence",
            "y_true": y_true,
            "y_pred": y_pred
        })
        preds_list.append(tmp)

    # --- Moving-average baseline: mean of last k days of 'prec' ---
    ma_series = df["prec"].rolling(window=ma_window,
                                   min_periods=ma_window).mean()

    for tgt in targets:
        y_true = df[tgt]
        y_pred = ma_series

        tmp = pd.DataFrame({
            "location": location_id,
            "date": df["date"],
            "target": tgt,
            "model": f"ma-{ma_window}",
            "y_true": y_true,
            "y_pred": y_pred
        })
        preds_list.append(tmp)

    preds = pd.concat(preds_list, ignore_index=True)

    # Drop rows with missing predictions (start of MA, or missing targets)
    preds = preds.dropna(subset=["y_true", "y_pred"])

    return preds

# ---------------------------------------------------------
# 3. Run baselines for all locations
# ---------------------------------------------------------

all_preds = []

for csv_path in files:
    preds_station = make_baseline_predictions_for_file(
        csv_path,
        targets=targets,
        ma_window=MA_WINDOW
    )
    all_preds.append(preds_station)

# Unified predictions for all locations and both baselines
preds_all = pd.concat(all_preds, ignore_index=True)


# ---------------------------------------------------------
# 4. Metrics function (RMSE / MAE)
# ---------------------------------------------------------

def compute_metrics(preds_df):
    """
    Compute RMSE and MAE for each (location, target, model).
    """
    def rmse(x):
        return np.sqrt(np.mean((x["y_pred"] - x["y_true"]) ** 2))

    def mae(x):
        return np.mean(np.abs(x["y_pred"] - x["y_true"]))

    metrics = (
        preds_df
        .groupby(["location", "target", "model"], as_index=False)
        .apply(lambda g: pd.Series({
            "RMSE": rmse(g),
            "MAE": mae(g),
            "n": len(g)
        }))
    )
    return metrics

metrics_baseline = compute_metrics(preds_all)

# ---------------------------------------------------------
# 5. Save predictions & metrics to ../prediction_results/
# ---------------------------------------------------------

results_dir = Path("../prediction_results")
results_dir.mkdir(parents=True, exist_ok=True)

# If you also want SEPARATE prediction files per baseline model:
for model_name, df_model in preds_all.groupby("model"):
    fname = results_dir / f"baseline_predictions_{model_name}.csv"
    df_model.to_csv(fname, index=False)
    print(f"Saved predictions for {model_name} to: {fname}")

# --- 7.2 Save metrics (RMSE/MAE) ---

# Separate metrics for ma-3 and persistence
metrics_ma3 = metrics_baseline[metrics_baseline["model"] == "ma-3"]
metrics_pers = metrics_baseline[metrics_baseline["model"] == "persistence"]

metrics_ma3_file = results_dir / "baseline_metrics_ma-3.csv"
metrics_pers_file = results_dir / "baseline_metrics_persistence.csv"

metrics_ma3.to_csv(metrics_ma3_file, index=False)
metrics_pers.to_csv(metrics_pers_file, index=False)

print(f"Saved MA-3 metrics to:        {metrics_ma3_file}")
print(f"Saved persistence metrics to: {metrics_pers_file}")



In [ ]:
# ---------------------------------------------------------
# 6. Standard plotting utilities (reusable for all models)
# ---------------------------------------------------------

def plot_pred_vs_true(preds_df, location_id=None, target=None):
    """
    Scatter plot of y_true vs y_pred for the selected location & target,
    using log1p scale on both axes (to spread out the cloud for skewed rain).

    Expects columns:
      ['location','date','target','model','y_true','y_pred'].
    """
    data = preds_df.copy()
    if location_id is not None:
        data = data[data["location"] == location_id]
    if target is not None:
        data = data[data["target"] == target]

    if data.empty:
        print("No data to plot for this selection.")
        return

    # log1p transform
    data = data.copy()
    data["y_true_log"] = np.log1p(data["y_true"])
    data["y_pred_log"] = np.log1p(data["y_pred"])

    plt.figure(figsize=(6, 6))
    sns.scatterplot(
        data=data,
        x="y_true_log",
        y="y_pred_log",
        hue="model",
        alpha=0.5
    )

    # 45-degree line in log space
    lims = [
        min(data["y_true_log"].min(), data["y_pred_log"].min()),
        max(data["y_true_log"].max(), data["y_pred_log"].max())
    ]
    plt.plot(lims, lims, "--", color="red", linewidth=1.5)
    plt.xlim(lims)
    plt.ylim(lims)
    plt.gca().set_aspect("equal", "box")

    # nicer tick labels in original mm units
    tick_vals = [0, 1, 2, 5, 10, 20, 50, 75]  # mm
    tick_locs = np.log1p(tick_vals)
    plt.xticks(tick_locs, tick_vals)
    plt.yticks(tick_locs, tick_vals)

    title_loc = location_id if location_id is not None else "all locations"
    title_tgt = target if target is not None else "all targets"
    plt.title(
        f"Predicted vs true precipitation (log1p scale)\n"
        f"{title_loc}, {title_tgt}"
    )
    plt.xlabel("True precipitation [mm]")
    plt.ylabel("Predicted precipitation [mm]")
    plt.legend(title="Model")
    plt.tight_layout()
    plt.show()


def plot_rmse_boxplot(metrics_df):
    """
    Boxplot of RMSE across locations, grouped by target and model.
    """
    plt.figure(figsize=(10, 5))
    sns.boxplot(
        data=metrics_df,
        x="target",
        y="RMSE",
        hue="model"
    )
    plt.title("RMSE across locations by target and model")
    plt.ylabel("RMSE")
    plt.xlabel("Target (forecast horizon)")
    plt.legend(title="Model")
    plt.tight_layout()
    plt.show()


# ---------------------------------------------------------
# 7. Example plots
# ---------------------------------------------------------

# Example 1: scatter for 1-day-ahead target at one location
example_location = preds_all["location"].iloc[0]
plot_pred_vs_true(preds_all, location_id=example_location, target="prec_1d_ahead")

# Example 2: RMSE boxplot across all locations
plot_rmse_boxplot(metrics_baseline)
